In [1]:
# %pip install "unstructured[md]" nltk langchain-text-splitters

from dotenv import load_dotenv

load_dotenv()

True

## **Single Document**

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain.document_loaders import TextLoader

# Load
markdown_path = "./Processed_Files_Introduction_to_End/CANopen_Integration_7012_V10_Mar11.md"
loader = TextLoader(markdown_path, autodetect_encoding=True)
doc = loader.load()
doc[0]

In [ ]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("###", "Header 3"),
    ("####", "Header 4"),
]

# MD splits
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=False
)
md_header_splits = markdown_splitter.split_text(doc[0].page_content)
md_header_splits

In [ ]:
# Char-level splits
chunk_size = 1000
chunk_overlap = 200
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(md_header_splits)
splits

## **All Documents**

In [2]:
import glob
import os

from langchain_core.documents import Document

In [79]:
# 1. Path/pattern for markdown files
folder_path = "./Processed_Files_Introduction_to_End/*.md"

# 2. Parameters for the Header Splitter
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    ("####", "Header 4"),
]
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, 
    strip_headers=False
)

# 3. Parameters for the character-level splitter
chunk_size = 1000
chunk_overlap = 200
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, 
    chunk_overlap=chunk_overlap
)

# Final list for all splits from all files
all_splits = []

for file_path in glob.glob(folder_path):
    # a) Load the file as raw text (preserving Markdown)
    loader = TextLoader(file_path, autodetect_encoding=True)
    docs = loader.load()  # usually returns a list [Document]
    
    # b) For each Document, perform two splits
    for doc in docs:
        md_header_splits = markdown_splitter.split_text(doc.page_content)
        splits = text_splitter.split_documents(md_header_splits)
        
        # c) Use the base file name as "name_file" (without the path and without extension)
        filename = os.path.splitext(os.path.basename(file_path))[0]

        for splitted_doc in splits:
            splitted_doc.metadata["name_file"] = filename
            # Optionally, remove the original "source" if not needed:
            # splitted_doc.metadata.pop("source", None)

        all_splits.extend(splits)

In [ ]:
min_size = 1000  # Minimum size for the merged chunks
merged_documents = []  # List to store merged documents
buffer_content = ""
buffer_metadata = {}

for doc in all_splits:
    text = doc.page_content.strip()  # remove extra spaces if necessary
    # If there is no content accumulated in the buffer and the chunk is small,
    # initialize the buffer with this chunk.
    if not buffer_content and len(text) < min_size:
        buffer_content = text
        buffer_metadata = doc.metadata
    # If there is already content in the buffer, concatenate it with the current chunk.
    elif buffer_content:
        buffer_content += "\n" + text  # add a line break to separate the texts
        # When the buffer reaches or exceeds the minimum size, create a Document
        if len(buffer_content) >= min_size:
            merged_documents.append(
                Document(page_content=buffer_content, metadata=buffer_metadata)
            )
            buffer_content = ""
            buffer_metadata = {}
    # If the current chunk is already large enough and there is nothing in the buffer, add it directly.
    elif len(text) >= min_size:
        merged_documents.append(Document(page_content=text, metadata=doc.metadata))

# If there is remaining content in the buffer, add it as well.
if buffer_content:
    merged_documents.append(Document(page_content=buffer_content, metadata=buffer_metadata))

# Example of printing the results:
for i, document in enumerate(merged_documents):
    print(f"Document {i+1}:")
    print("Metadata:", document.metadata)
    print("Content (first 200 characters):", document.page_content[:200])
    print("-" * 50)

In [ ]:
print(f"Generated {len(all_splits)} chunks in total.")

In [ ]:
print(f"Generated {len(merged_documents)} chunks in total.")

In [ ]:
merged_documents

# **RAG**

In [3]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain_ollama import ChatOllama
from typing_extensions import TypedDict
from typing import List
from langchain.schema import Document
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langgraph.graph import START, StateGraph
from IPython.display import Image, display
from pprint import pprint
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings

In [5]:
# Example embeddings (you can use OpenAIEmbeddings or another)
# embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Create an in-memory local vector store (FAISS)
# vector_store = FAISS.from_documents(merged_documents, embedding=embeddings)

/tmp/ipykernel_8862/3728433904.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


In [6]:
vector_store = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [7]:
# Example of an LLM
# llm = ChatOllama(temperature=0, model="gemma3:12b")

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

/tmp/ipykernel_8862/2879934330.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")


In [8]:
template = """You are an expert assistant for answering questions accurately and concisely based on the provided context.

### Instructions:
- Answer in a direct and clear manner.
- Answer **in the same language as the question**.
- Focus on **the purpose or main function** of the topic in question.
- Avoid unnecessary details unless explicitly relevant.
- Start with **a direct answer**, then add clarification only if needed.

### Example:
Q: Why is a library important in a community?
A: A library provides access to books, knowledge, and resources that support education and lifelong learning. It also serves as a community hub for research and collaboration.

Q: What is the purpose of an ecosystem?
A: An ecosystem maintains balance in nature by allowing different species to interact and support each other. It regulates climate, provides food sources, and sustains biodiversity.

### Now answer the following:

Question: {question}

Context: {context}

Answer in the same language as the user's input question.

Answer (in the language of the question):  
"""

prompt = ChatPromptTemplate.from_template(template)

# llm = ChatOllama(temperature=0, model="qwen2:7b-instruct")

In [9]:
question = "Qual é o objetivo de um dispositivo Linking na integração de CANopen com PROFINET IO?"

In [10]:
def generate_hypothetical_document(question: str) -> str:
    """
    Generates a hypothetical, technically accurate document in English for the given question,
    to be used in vector-based retrieval systems. The result must be focused, factual, and domain-specific.
    """
    hyde_template = (
        "You are an AI assistant specialized in industrial automation, communication protocols, and control networks. "
        "Your task is to generate a detailed and technically accurate hypothetical answer that could appear in a knowledge base, "
        "technical documentation, or white paper related to industrial networking. Follow these strict instructions:\n"
        "1. The response must ONLY contain the hypothetical document text—no commentary, notes, explanations, or formatting.\n"
        "2. The content must be written in English and must not include greetings or apologies.\n"
        "3. The text should be informative, concise, and focused on answering the question using domain-relevant terminology.\n"
        "4. Do not restate the question. Just provide the content.\n\n"
        "Question: {question}\n\n"
        "Hypothetical Document:"
    )

    prompt_hyde = ChatPromptTemplate.from_template(hyde_template)
    messages = prompt_hyde.format(question=question)

    # hyde_llm = ChatOllama(model="gemma3:12b", temperature=0.7)
    hyde_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

    # Generate the hypothetical document
    response = hyde_llm.invoke(messages)

    # Extract and clean content
    if hasattr(response, "content"):
        return response.content.strip()
    else:
        return response.strip()


In [11]:
generate_hypothetical_document(question)

'The Linking device serves as a critical component in the integration of CANopen with PROFINET IO, facilitating seamless communication between these two distinct network protocols. Its primary objective is to act as a bridge, enabling data exchange and interoperability between the CANopen field devices and PROFINET IO controllers.\n\nBy converting CANopen messages into PROFINET IO frames and vice versa, the Linking device allows for the transparent transmission of process data, service data, and network management commands. This conversion is essential for maintaining the integrity and real-time capabilities of both networks, ensuring that devices on CANopen can effectively communicate with PROFINET IO systems without losing the benefits of either protocol.\n\nAdditionally, the Linking device may implement specific functionalities such as protocol translation, data mapping, and synchronization of cyclic and acyclic data flows. This includes handling the timing requirements inherent in 

In [12]:
retriever = vector_store.as_retriever()

In [16]:
class State(TypedDict):
    question: str           # User's question
    hypothetical_answer: str # Hypothetical document generated for retrieval
    context: List[Document] # Documents returned by the retrieve function
    answer: str             # Final answer from the LLM

def generate_hypothetical_document(question: str) -> str:
    """
    Generates an ideal hypothetical answer for the given question to be used for document retrieval.
    The answer must be solely the hypothetical document, with no additional commentary, explanations, or formatting.
    """
    hyde_template = (
        "You are an expert assistant. Given the following question, generate a comprehensive hypothetical answer "
        "that will serve as an ideal document for information retrieval. Follow these strict rules:\n"
        "1. Do not include any extra commentary, headers, or notes. Your response must be only the hypothetical document text.\n"
        "2. Do not include greetings, apologies, or any text before or after the answer.\n"
        "3. The answer should be concise, factual, and focused solely on answering the question.\n\n"
        "Question: {question}\n\n"
        
        "Hypothetical Answer (in english. ONLY the text, nothing else):"
    )
    prompt_hyde = ChatPromptTemplate.from_template(hyde_template)
    messages = prompt_hyde.format(question=question)

    # hyde_llm = ChatOllama(model="gemma3:12b", temperature=0.7)
    hyde_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
    
    # Utilize the LLM to generate the hypothetical document.
    response = hyde_llm.invoke(messages)
    
    # If the response is wrapped in an AI message, extract its content.
    if hasattr(response, "content"):
        return {"hypothetical_answer": response.content.strip()}
    else:
        return {"hypothetical_answer": response.strip()}

def retrieve(state: State):
    """
    Retrieval step: performs a similarity search in the vector_store
    and returns the documents that will serve as context for the answer.
    
    Args:
        state (State): The current state containing the user's question.
    
    Returns:
        dict: A dictionary with the key "context" that will be added to the state.
    """
    retrieved_docs = retriever.invoke(state["hypothetical_answer"])
    # Returns a dictionary with the key "context"
    return {"context": retrieved_docs}

def generate(state: State):
    """
    Generation step: formats the 'context' and the 'question' into a prompt,
    calls the LLM, and obtains the final answer.
    
    Args:
        state (State): The current state containing the user's question and the retrieved context.
    
    Returns:
        dict: A dictionary with the key "answer" containing the final answer from the LLM.
    
    Raises:
        ValueError: If the response format from the LLM is unexpected.
    """
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    
    response = llm.invoke(messages)  # or llm(messages), depending on your setup

    # If the response is of type AIMessage and has the attribute 'content', use it:
    if hasattr(response, "content"):
        return {"answer": response.content}
    elif isinstance(response, str):
        return {"answer": response}
    else:
        raise ValueError("Unexpected response format.")

In [17]:
# Build the application graph:
graph_builder = StateGraph(State).add_sequence([generate_hypothetical_document, retrieve, generate])
graph_builder.add_edge(START, "generate_hypothetical_document")
graph_builder.add_edge("generate_hypothetical_document", "retrieve")
graph_builder.add_edge("retrieve", "generate")

# Compile into a callable "graph" object
graph = graph_builder.compile()

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

## **Queries**

In [ ]:
import re
from pprint import pprint

def clean_answer(answer: str) -> str:
    """
    Cleans the generated answer by removing unnecessary information and formatting.
    
    Args:
        answer (str): The raw answer generated by the LLM.
        
    Returns:
        str: The cleaned answer, formatted for better readability.
    """

    # Remove logs such as INFO:langchain...
    cleaned = re.sub(r'INFO:.*?(\n|$)', '', answer)

    # Extract the value of the 'answer' key if it's a dictionary string
    match = re.search(r"'answer':\s*([\"'])(.*?)\1", cleaned, re.DOTALL)
    if match:
        cleaned = match.group(2)

    # Remove markdown bold formatting **text**
    cleaned = re.sub(r"\*\*(.*?)\*\*", r"\1", cleaned)

    # Remove duplicate spaces and normalize line breaks
    cleaned = re.sub(r'\n{3,}', '\n\n', cleaned)
    cleaned = re.sub(r'[ \t]+$', '', cleaned, flags=re.MULTILINE)

    return cleaned.strip()

def clean_context(context):
    """
    Simplifies the context by extracting relevant metadata and limiting content length.
    
    Args:
        context (List[Document]): The list of documents retrieved as context.
        
    Returns:
        List[dict]: A simplified list of dictionaries containing the document ID and a snippet of the content.
    """
    
    return [
        {
            "id": doc.metadata.get("name_file", doc.metadata.get("source", "unknown")),
            "content": doc.page_content[:300] + "..." if len(doc.page_content) > 300 else doc.page_content
        }
        for doc in context
    ]

def process_question(question: str):
    """
    Process a user's question by retrieving context and generating an answer using the graph.

    Args:
        question (str): The user's question.

    Prints:
        The retrieved context and the generated answer.
    """
    for step_result in graph.stream({"question": question}, stream_mode="updates"):
        if "retrieve" in step_result:
            context = step_result.get("retrieve", {}).get("context", [])
            simplified_context = clean_context(context)
            print("\n📚 Retrieved Context:\n")
            pprint(simplified_context)

        if "generate" in step_result:
            raw_answer = step_result.get("generate", {}).get("answer", "")
            formatted_answer = clean_answer(raw_answer)
            print("\n🧠 Generated Answer:\n")
            print(formatted_answer)

### 1 - List the main causes of reflections in Profibus cables.

In [ ]:
question_1 = "List the main causes of reflections in Profibus cables"
process_question(question_1)


📚 Retrieved Context:

[{'content': '## **PROFIBUS cable too long**  \n'
             'Cable that is too long generally functions like a condenser. It '
             'changes the signal form. The result is that for a square wave '
             'signal the rising edge is rounded (e-function). This effect is '
             'more pronounced the longer the PROFIBUS cable.  \n'
             'If the signal is changed too ...',
  'id': 'Copia di PROFIBUS_Commissioning_8032_V123_Sep22 (1)'},
 {'content': '## **Defective Bus Terminator**  \n'
             'For the bus connection two types of errors can occur, both of '
             'which cause a signal reflection. The reflection strength depends '
             'on the error.  \n'
             'For the one error, too many terminators are switched on. Thus '
             'approximately 1/3 of the signal is reflected and is turned by...',
  'id': 'Copia di PROFIBUS_Commissioning_8032_V123_Sep22 (1)'},
 {'content': '### <span id="page-24-1"></span

### 2 - What are the different interference coupling mechanisms in Profibus cables? What measures can be taken to reduce each type of interference?

In [39]:
question_2 = "What are the different interference coupling mechanisms in Profibus cables? What measures can be taken to reduce each type of interference?"
process_question(question_2)


📚 Retrieved Context:

[{'content': '### <span id="page-24-1"></span>**2.1.4 Radiated coupling**  \n'
             'In industrial cables of usually up to 100 m of length, radiated '
             'coupling between high-energy disturbers and signal current '
             'circuits only occurs at high frequencies (approx. 30 MHz and '
             'higher). This type of coupling is caused by the electr...',
  'id': 'Copia di Earthing-Shielding_8102_V31_Sep22'},
 {'content': '## <span id="page-19-0"></span>**2.1 Couplings**  \n'
             'A source of disturbance is only capable of disturbing another '
             'device if there are coupling lines. The coupling lines connect '
             'the source to the susceptible device (see [Figure '
             '2.2\\)](#page-19-1). In this context, the term "susceptible '
             'device" refers ...',
  'id': 'Copia di Earthing-Shielding_8102_V31_Sep22'},
 {'content': 'Chapter [3.1.2](#page-28-0) described active shielding, which '
  

### 3 - In which situations should spur lines not be used in Profibus DP network topology?

In [40]:
question_3 = "In which situations should spur lines not be used in Profibus DP network topology?"
process_question(question_3)


📚 Retrieved Context:

[{'content': '## **2.1.3.3 Topologies of PROFIBUS-MBP (PA)**  \n'
             '<span id="page-42-1"></span>With PROFIBUS-MBP (PA) the slaves '
             'are powered via the bus. As a result, copper cables have to be '
             'used as the transmission medium.  \n'
             'For PROFIBUS-MBP (PA) line and tree topologies as well as a '
             'combination of these are permis...',
  'id': 'Copia di PROFIBUS_Design_Guideline_8012_V129_Sep20 (1)'},
 {'content': '## <span id="page-51-0"></span>**5.2 Checking the Cabling '
             'Infrastructure**  \n'
             'Checking the cabling infrastructure should be one of your first '
             'troubleshooting steps. Errors can creep in here, for example, if '
             'the routing of the PROFIBUS cable has been changed. The '
             'potential for errors when extending or...',
  'id': 'Copia di PROFIBUS_Commissioning_8032_V123_Sep22 (1)'},
 {'content': '- The maximum trunk cable len

### 4 - Determine the minimum separation distance between a Profibus cable and the following cables:
a. AS-Interface type Profibus cable

b. 24V power cable for a digital output module

c. Single-phase 240V AC power cable for supplying a DC power supply

d. Grounding equipotential bonding cable between cabinets

e. Three-phase 380V power cable for a Profibus frequency inverter

In [ ]:
question_4 = """Determine the minimum separation distance between a Profibus cable and the following cables:

a. AS-Interface type Profibus cable

b. 24V power cable for a digital output module

c. Single-phase 240V AC power cable for supplying a DC power supply

d. Grounding equipotential bonding cable between cabinets

e. Three-phase 380V power cable for a Profibus frequency inverter"""
process_question(question_4)


📚 Retrieved Context:

[{'content': '![](_page_132_Picture_6.jpeg)  \n'
             'For more details about minimum separation distances please refer '
             'to the IEC 61784-5-3 or EN 50174-2 standard, respectively.  \n'
             '![](_page_132_Picture_8.jpeg)  \n'
             'If possible, use only conducting cable ducts and plan for '
             'regular connections to a functional equipotential bo...',
  'id': 'Copia di PROFIBUS_Design_Guideline_8012_V129_Sep20 (1)'},
 {'content': '#### **General notes on cable routing inside buildings**  \n'
             'In general, power supply cables and communication cables shall '
             'be laid separately.  \n'
             'You should first check where the power supply cables are laid or '
             'planned to be laid in the plant. In practice, it may be '
             'impossible to completely separate t...',
  'id': 'Copia di PROFIBUS_Design_Guideline_8012_V129_Sep20 (1)'},
 {'content': '## **Use for potential s

### 5 - Two control rooms located at opposite ends of an area must share the same Profibus network. Each room has its own grounding system, and there is a significant potential difference between them. List three methods to install the network in a way that avoids issues caused by separate ground potentials.

In [ ]:
question_5 = "Two control rooms located at opposite ends of an area must share the same Profibus network. Each room has its own grounding system, and there is a significant potential difference between them. List three methods to install the network in a way that avoids issues caused by separate ground potentials."
process_question(question_5)


📚 Retrieved Context:

[{'content': 'The hybrid plug systems also include a variant for transmitting '
             'data over fiber-optic cable fibers and 24 V operating voltage '
             'for the peripheral devices via copper cables in a single hybrid '
             'cable.  \n'
             'Experience shows that difficulties with the transmission '
             'technology in PROFIBUS networks can most oft...',
  'id': 'Copia di PROFIBUS_Systembeschreibung_ENG_web'},
 {'content': 'The potential separation between the two input channels and '
             'against the device ground is particularly important. Separation '
             'of the two channels ensures that they do not affect each other. '
             'It is just as important that both channels are also potential '
             'separated from the device ground. If this is not t...',
  'id': 'Copia di PROFIBUS_Commissioning_8032_V123_Sep22 (1)'},
 {'content': 'The following subsections provide general explanations an

### 6 - Discuss the factors that have a significant effect on the overall cycle time of a DP network.

In [ ]:
question_6 = "Discuss the factors that have a significant effect on the overall cycle time of a DP network."
process_question(question_6)


📚 Retrieved Context:

[{'content': 'The communication between the master and a slave always includes '
             'the transmission of two telegrams into which the process data '
             '(input and output data) is packaged. The amount of process data '
             'to be transmitted depends on the input and output data of the '
             'slaves. Intelligent devices usually have betwee...',
  'id': 'Copia di PROFIBUS_Design_Guideline_8012_V129_Sep20 (1)'},
 {'content': '#### <span id="page-56-0"></span>**Figure 2-12: PROFIBUS bus '
             'cycle times**  \n'
             '(PA)  \n'
             'Compared to the bus cycle time of PROFIBUS DP, the bus cycle '
             'time of PROFIBUS-MBP (PA) is quite long. However, this does not '
             'necessarily mean that the total cycle time of a line consisting '
             'of both PROFIBUS DP and PRO...',
  'id': 'Copia di PROFIBUS_Design_Guideline_8012_V129_Sep20 (1)'},
 {'content': '## **PROFIBUS DP**  \n'
   

### 7 - A PA segment is used to wire 24 slave devices in a non-hazardous area. Determine the maximum spur length when:
a. Each spur carries only one slave,

b. Each spur carries four slaves.

What is the maximum length of the main trunk cable in each case?

In [ ]:
question_7 = """A PA segment is used to wire 24 slave devices in a non-hazardous area. Determine the maximum spur length when:
a. Each spur carries only one slave,

b. Each spur carries four slaves.

What is the maximum length of the main trunk cable in each case?"""
process_question(question_7)


📚 Retrieved Context:

[{'content': '- The maximum trunk cable length is 1 000 m.\n'
             '- A trunk with connected spurs must be used to realize the '
             'topology.\n'
             "- Line type 'A' must be used.  \n"
             'With DART the main bus (trunk) is also intrinsically safe and '
             'can be accessed during operation without the glow of a fire. '
             'Sufficient power for realizing ...',
  'id': 'Copia di PROFIBUS_Design_Guideline_8012_V129_Sep20 (1)'},
 {'content': '## **2.1.3.3 Topologies of PROFIBUS-MBP (PA)**  \n'
             '<span id="page-42-1"></span>With PROFIBUS-MBP (PA) the slaves '
             'are powered via the bus. As a result, copper cables have to be '
             'used as the transmission medium.  \n'
             'For PROFIBUS-MBP (PA) line and tree topologies as well as a '
             'combination of these are permis...',
  'id': 'Copia di PROFIBUS_Design_Guideline_8012_V129_Sep20 (1)'},
 {'content': '- Spurs 

### 8 - Does PA wiring have to use screened twisted pair cable?

In [ ]:
question_8 = "Does PA wiring have to use screened twisted pair cable?"
process_question(question_8)


📚 Retrieved Context:

[{'content': 'A two-core shielded cable is used for the transmission medium. '
             'The important thing here is that different cables are involved '
             'for PROFIBUS DP (cable Type A according to IEC 61158 Part 2 '
             'Interface RS485(-IS)) and PROFIBUS PA (cable Type A according to '
             'IEC 61158 Part 2 Interface MBP(-IS)). (See Table ...',
  'id': 'Copia di PROFIBUS_Systembeschreibung_ENG_web'},
 {'content': 'When connecting the nodes, ensure that the data cables are not '
             'mixed up. To achieve high interference resistance of the system '
             'against electromagnetic radiation, a shielded data cable (type A '
             'is shielded) should definitely be used. The shielding is to be '
             'connected to the protective ground on both si...',
  'id': 'Copia di PROFIBUS_Systembeschreibung_ENG_web'},
 {'content': '#### <span id="page-112-0"></span>**Table 3-8: Parameters of '
             'line

### 9 - A non-hazardous plant is currently fitted with 4 to 20 mA instrumentation, wired using a mixture of single and multi-core cable. What are the main considerations when considering moving over to a PROFIBUS PA system?

In [ ]:
question_9 = "A non-hazardous plant is currently fitted with 4 to 20 mA instrumentation, wired using a mixture of single and multi-core cable. What are the main considerations when considering moving over to a PROFIBUS PA system?"
process_question(question_9)


📚 Retrieved Context:

[{'content': '#### **Fig. 24: Stepwise introduction of PROFINET in the '
             'intrinsically safe field**\n'
             '#### **PROFIBUS PA remains a key technology**  \n'
             'PROFIBUS PA enables long cable distances and explosion '
             'protection in process automation as well as the native, digital '
             'integration of field instrumentation in control s...',
  'id': 'Copia di PROFIBUS_Systembeschreibung_ENG_web'},
 {'content': '## <span id="page-35-0"></span>**3.3 Acceptance Measurements - '
             'PROFIBUS MBP (PA)**  \n'
             'The PROFIBUS MBP (PA) wiring can be checked with a multimeter or '
             'installation tester. Chapter [6.2.3](#page-89-1) describes how '
             'to do this. A checklist for the acceptance measurements for '
             'PROFIBUS MBP (PA) can be fo...',
  'id': 'Copia di PROFIBUS_Commissioning_8032_V123_Sep22 (1)'},
 {'content': 'A two-core shielded cable is used for t

### 10 - Determine the maximum trunk and spur cable lengths for a non-intrinsically safe MBP segment with 20 slaves each connected via the maximum possible spur length. Estimate the required coupler current rating if each slave device takes a maximum of 14 mA.

In [ ]:
question_10 = "Determine the maximum trunk and spur cable lengths for a non-intrinsically safe MBP segment with 20 slaves each connected via the maximum possible spur length. Estimate the required coupler current rating if each slave device takes a maximum of 14 mA."
process_question(question_10)


📚 Retrieved Context:

[{'content': '## **3.2.3.1 Instructions for PROFIBUS MBP (PA) planning**  \n'
             '- Check the spur lengths according to [Table 3-4](#page-97-1) '
             'and observe the maximum permissible length.\n'
             '- The current consumption of the connected nodes and device '
             'couplers must not exceed the maximum permissible supply current '
             'of the bu...',
  'id': 'Copia di PROFIBUS_Design_Guideline_8012_V129_Sep20 (1)'},
 {'content': '- The maximum trunk cable length is 1 000 m.\n'
             '- A trunk with connected spurs must be used to realize the '
             'topology.\n'
             "- Line type 'A' must be used.  \n"
             'With DART the main bus (trunk) is also intrinsically safe and '
             'can be accessed during operation without the glow of a fire. '
             'Sufficient power for realizing ...',
  'id': 'Copia di PROFIBUS_Design_Guideline_8012_V129_Sep20 (1)'},
 {'content': '![](_pag